In [1]:
import pandas as pd

def unique_list(x):
    return sorted({v for v in x if pd.notna(v)})

In [2]:
pd.read_parquet("dados\passageiros_unificado.parquet")

,data,tipo,area,empresa,linha,passageiros_dinheiro,passageiros_comum,passageiros_comum_mensal,passageiros_estudante,passageiros_estudante_mensal,...,passageiros_integracao_onibus,passageiros_gratuitos,passageiros_gratuitos_estudante,passageiros_total,arquivo,grupo,lote,passageiros_vale_transporte,route_id,passageiros_integracao
0,2019-02-01,CONCESSAO,AREA 1,GATO PRETO,N10511 - TERM CACHOEIRINHA/TERM LAPA,28,57.0,9,2,0,...,162,27,0.0,331,dados\passageiros\2019\01-FEV.xls,None,None,NaN,N10511,166.0
1,2019-02-01,CONCESSAO,AREA 1,GATO PRETO,N10611 - T PQ D PEDRO /MET BARRA FUNDA,0,3.0,0,0,0,...,1,0,0.0,5,dados\passageiros\2019\01-FEV.xls,None,None,NaN,N10611,1.0
2,2019-02-01,CONCESSAO,AREA 1,GATO PRETO,N14311 - METRO BARRA FUNDA/MORRO GRANDE,4,10.0,2,0,0,...,20,7,0.0,67,dados\passageiros\2019\01-FEV.xls,None,None,NaN,N14311,23.0
3,2019-02-01,CONCESSAO,AREA 1,GATO PRETO,800001 - TERM LAPA/PCA RAMOS DE AZEVEDO,473,1498.0,208,35,0,...,2541,1526,41.0,7383,dados\passageiros\2019\01-FEV.xls,None,None,NaN,800001,2790.0
4,2019-02-01,CONCESSAO,AREA 1,GATO PRETO,830001 - TERM PIRITUBA/TERM LAPA,47,193.0,24,4,0,...,381,187,11.0,989,dados\passageiros\2019\01-FEV.xls,None,None,NaN,830001,419.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5056907,2017-01-31,PERMISSAO,AREA 8,TRANSCAP,807P10 - PARAISOPOLIS/SANT0 AMARO,218,1608.0,259,5,0,...,1868,393,0.0,5206,dados\passageiros\2018\Passag-20180131.xls,None,None,NaN,807P10,1980.0
5056908,2017-01-31,PERMISSAO,AREA 8,TRANSCAP,807210 - PQ IPE/BUTANTA,618,2074.0,350,27,0,...,2269,1042,0.0,9002,dados\passageiros\2018\Passag-20180131.xls,None,None,NaN,807210,3443.0
5056909,2017-01-31,PERMISSAO,AREA 8,TRANSCAP,807310 - JD GUARAU/BUTANTA,428,961.0,117,15,0,...,1095,550,2.0,4367,dados\passageiros\2018\Passag-20180131.xls,None,None,NaN,807310,1587.0
5056910,2017-01-31,PERMISSAO,AREA 8,TRANSCAP,807810 - JD DAS PALMAS/METRO CAMPO LIMP,317,1566.0,256,25,0,...,1885,1043,0.0,6484,dados\passageiros\2018\Passag-20180131.xls,None,None,NaN,807810,2200.0


In [2]:
df_routes=pd.read_parquet(r"dados\routes_historico.parquet")
df_trips=pd.read_parquet(r'dados\trips_historico.parquet')
df_stops=pd.read_parquet(r'dados\stops_historico_categorizado.parquet')
df_stops_times=pd.read_parquet(r'dados\stop_times_historico.parquet')
df_frequencies=pd.read_parquet(r"dados\frequencies_historico.parquet")

In [3]:
df_max_trip=df_trips.groupby("route_id")['data_referencia'].max().reset_index()
df_max_trip=df_trips.merge(df_max_trip,on=['route_id','data_referencia'],how='inner')
df_max_trip.to_parquet(r"dados\trips_more_recent.parquet",index=False)

Informações da tabela de rotas

In [4]:
df_routes_categorizado= (
    df_routes
    .groupby("route_id")
    .agg(
        route_long_name=("route_long_name", unique_list),
        route_color=("route_color", lambda x: unique_list(x)[0] if len(unique_list(x))>0 else None),
        route_text_color=("route_text_color", lambda x: unique_list(x)[0] if len(unique_list(x))>0 else None),
        datas_operacao=("data_referencia", unique_list)
    )
    .reset_index()
)

Informações da tabela de trips

In [5]:
df_trips_agrupado = (
    df_trips
    .groupby("route_id")
    .agg(
        direcoes=("direction_id", unique_list),
        service_ids=("service_id", unique_list),    )
    .reset_index()
)
df_trips_agrupado['direcoes']=df_trips_agrupado['direcoes'].apply(lambda x: "Circular" if len(x) == 1 else "Não circular")

df_routes_categorizado=df_routes_categorizado.merge(df_trips_agrupado,on='route_id')

Informações da tabela de frequências

In [6]:
df_horarios=df_frequencies.copy()
df_frequencies["start_time"] = pd.to_datetime(df_frequencies["start_time"], format="%H:%M:%S")
df_frequencies["end_time"] = pd.to_datetime(df_frequencies["end_time"], format="%H:%M:%S")

In [7]:
linhas_noturnas=df_frequencies[df_frequencies.start_time.astype(str).str.contains("03:00:00")].trip_id.apply(lambda x: "".join(x.split("-")[0:2])).to_list()
df_routes_categorizado['noturna']=df_routes_categorizado['route_id'].isin(linhas_noturnas)

In [8]:
df_horario_operacao=df_frequencies.copy()
df_horario_operacao['route_id']=df_horario_operacao.trip_id.apply(lambda x: "".join(x.split("-")[0:2]))
df_horario_operacao=df_horario_operacao.drop_duplicates(subset=['route_id','start_time','end_time','data_referencia'])
df_horario_operacao["horas_operacao"]=(df_frequencies["end_time"]-df_horario_operacao['start_time']).dt.total_seconds() / 3600
df_horario_operacao=df_horario_operacao.groupby(["data_referencia",'route_id'])['horas_operacao'].sum().reset_index()
df_horario_operacao_max=df_horario_operacao.merge(df_max_trip[['route_id','data_referencia']],on=['route_id','data_referencia'],how='inner').drop_duplicates()

df_routes_categorizado=df_routes_categorizado.merge(df_horario_operacao_max[['route_id','horas_operacao']],on='route_id')
df_horario_operacao.to_parquet(r"dados\frequencies_horario_operacao_historico.parquet",index=False)

In [88]:

df_horarios['headway_secs']=df_horarios['headway_secs'] // 60

In [89]:
df_horarios['horarios_operacao']=df_horarios['start_time'].astype(str).apply(lambda x: "h".join(x.split(":")[:2])) + " - " + df_horarios['end_time'].astype(str).apply(lambda x: "h".join(x.split(":")[:2]))
df_horarios=df_horarios[['data_referencia','horarios_operacao','headway_secs','trip_id']].drop_duplicates().pivot(index=['data_referencia','trip_id'],columns='horarios_operacao',values='headway_secs').reset_index()

In [90]:
df_horarios

horarios_operacao,data_referencia,trip_id,00h00 - 00h59,01h00 - 01h59,02h00 - 02h59,03h00 - 03h59,04h00 - 04h59,05h00 - 05h59,06h00 - 06h59,07h00 - 07h59,...,14h00 - 14h59,15h00 - 15h59,16h00 - 16h59,17h00 - 17h59,18h00 - 18h59,19h00 - 19h59,20h00 - 20h59,21h00 - 21h59,22h00 - 22h59,23h00 - 23h59
0,201501,1015-10-0,NaN,NaN,NaN,NaN,NaN,20.0,20.0,20.0,...,60.0,30.0,30.0,20.0,20.0,30.0,30.0,60.0,60.0,NaN
1,201501,1016-10-0,30.0,NaN,NaN,NaN,60.0,8.0,4.0,4.0,...,6.0,6.0,5.0,6.0,6.0,6.0,10.0,10.0,10.0,20.0
2,201501,1016-10-1,15.0,NaN,NaN,NaN,NaN,20.0,10.0,10.0,...,6.0,6.0,6.0,5.0,6.0,6.0,6.0,10.0,10.0,10.0
3,201501,1017-10-0,60.0,NaN,NaN,60.0,10.0,5.0,5.0,6.0,...,8.0,8.0,6.0,12.0,7.0,7.0,12.0,12.0,20.0,30.0
4,201501,1017-10-1,20.0,NaN,NaN,NaN,30.0,15.0,6.0,7.0,...,7.0,7.0,6.0,5.0,5.0,6.0,6.0,12.0,12.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276255,202503,N839-11-0,60.0,60.0,60.0,60.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
276256,202503,N840-11-0,60.0,60.0,60.0,60.0,60.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
276257,202503,N841-11-0,60.0,60.0,60.0,60.0,60.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
276258,202503,N842-11-0,60.0,60.0,60.0,60.0,60.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Informaçõse da tabela de paradas

In [91]:
df_stops=df_stops[['stop_id','Regiões','Subprefeituras ','Distritos','CD_SETOR','v0001','nome_estacao_mais_proxima_existente','dist_estacao_mais_proxima_existente','nome_estacao_mais_proxima','dist_estacao_mais_proxima']].drop_duplicates()
df_stops_infos=df_stops.merge(df_trips[['route_id','stop_id']],on='stop_id').drop(columns='stop_id').drop_duplicates()

In [92]:
df_infos_geograficas=(df_stops_infos
    .groupby("route_id")
    .agg(
        regioes=("Regiões", unique_list),
        subprefeituras=("Subprefeituras ", unique_list),
        bairros=("Distritos", unique_list),
    )
    .reset_index()
)

df_infos_populacao=df_stops_infos.drop_duplicates(subset=['route_id','CD_SETOR'])[['route_id','v0001']].groupby("route_id").agg(populacoes=("v0001",sum)).reset_index()

df_infos_estacoes=df_stops_infos[['route_id','nome_estacao_mais_proxima_existente','nome_estacao_mais_proxima']]
df_infos_estacoes['beneficiado_por_nova_estacao']=df_infos_estacoes['nome_estacao_mais_proxima_existente']!=df_infos_estacoes['nome_estacao_mais_proxima']
df_infos_estacoes['estacao_nova_no_bairro']=(df_infos_estacoes['beneficiado_por_nova_estacao']==True)&(df_infos_estacoes['nome_estacao_mais_proxima_existente'].isna())

df_infos_estacoes=(df_infos_estacoes
    .groupby("route_id")
    .agg(
        estacoes_proximas_existentes=("nome_estacao_mais_proxima_existente", unique_list),
        estacoes_proximas=("nome_estacao_mais_proxima", unique_list),
        beneficiado_por_nova_estacao=("beneficiado_por_nova_estacao", lambda x: True if True in (unique_list(x)) else False),
        estacao_nova_no_bairro=("estacao_nova_no_bairro", lambda x: True if True in (unique_list(x)) else False),
    )
    .reset_index()
)

df_infos_geograficas=df_infos_geograficas.merge(df_infos_populacao,on='route_id')
df_infos_geograficas=df_infos_geograficas.merge(df_infos_estacoes,on='route_id')


C:\Users\franc\AppData\Local\Temp\ipykernel_18704\762652553.py:11: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df_infos_populacao=df_stops_infos.drop_duplicates(subset=['route_id','CD_SETOR'])[['route_id','v0001']].groupby("route_id").agg(populacoes=("v0001",sum)).reset_index()
C:\Users\franc\AppData\Local\Temp\ipykernel_18704\762652553.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_infos_estacoes['beneficiado_por_nova_estacao']=df_infos_estacoes['nome_estacao_mais_proxima_existente']!=df_infos_estacoes['nome_estacao_mais_proxima']
C:\Users\franc\AppDa

In [93]:
df_routes_categorizado=df_routes_categorizado.merge(df_infos_geograficas,on='route_id')

Linha bairro-centro, bairro-bairro, bairro-Metrô

In [94]:
bairros_centro_expandido = {
    "Sé", "República", "Bela Vista", "Consolação", "Santa Cecília",
    "Liberdade", "Cambuci", "Bom Retiro", "Brás", "Pari", "Moóca",
    "Belém", "Tatuapé", "Água Rasa", "Vila Prudente", "Ipiranga",
    "Vila Mariana", "Saúde", "Moema", "Itaim Bibi", "Jardim Paulista",
    "Pinheiros", "Alto de Pinheiros", "Lapa", "Perdizes", "Barra Funda"
}

def categorizar_linha(bairros, estacoes, bairros_centro):
    # garante que bairros seja lista
    bairros = bairros if isinstance(bairros, list) else []
    estacoes = estacoes if isinstance(estacoes, list) else []
    
    # caso 1: bairro-centro (toca pelo menos um bairro do centro)
    if any(b in bairros_centro for b in bairros):
        return "Bairro-Centro"
    
    # caso 2: bairro-metrô (não passa no centro, mas passa por metrô)
    if len(estacoes) > 0:
        return "Bairro-Metrô/CPTM"
    
    # caso 3: bairro-bairro
    return "Bairro-Bairro"

df_routes_categorizado["tipo_de_rota_atual"] = df_routes_categorizado.apply(
    lambda row: categorizar_linha(row["bairros"], row["estacoes_proximas_existentes"], bairros_centro_expandido),
    axis=1
)

df_routes_categorizado["tipo_de_rota_futuro"] = df_routes_categorizado.apply(
    lambda row: categorizar_linha(row["bairros"], row["estacoes_proximas"], bairros_centro_expandido),
    axis=1
)

In [95]:
df_routes_categorizado['datas_operacao']=df_routes_categorizado['datas_operacao'].apply(lambda x: f"{min(x)} - {max(x)}")

Converte tipos de lista para string

In [96]:
columns_list=['route_long_name','service_ids','regioes','subprefeituras','bairros','estacoes_proximas_existentes','estacoes_proximas']
for c in columns_list:
    df_routes_categorizado[c]=df_routes_categorizado[c].apply(lambda x: ", ".join(x))

In [97]:
df_routes_categorizado.columns

Index(['route_id', 'route_long_name', 'route_color', 'route_text_color',
       'datas_operacao', 'direcoes', 'service_ids', 'noturna',
       'horas_operacao', 'regioes', 'subprefeituras', 'bairros', 'populacoes',
       'estacoes_proximas_existentes', 'estacoes_proximas',
       'beneficiado_por_nova_estacao', 'estacao_nova_no_bairro',
       'tipo_de_rota_atual', 'tipo_de_rota_futuro'],
      dtype='object')

In [98]:
df_routes_categorizado=df_routes_categorizado[['route_id', 'route_long_name',
       'datas_operacao', 'direcoes', 'service_ids', 'noturna',
       'horas_operacao', 'regioes', 'subprefeituras', 'bairros', 'populacoes',
       'estacoes_proximas_existentes', 'estacoes_proximas',
       'beneficiado_por_nova_estacao', 'estacao_nova_no_bairro',
       'tipo_de_rota_atual', 'tipo_de_rota_futuro', 'route_color', 'route_text_color',]]

Salva em parquet

In [99]:
df_horarios.to_parquet(r"dados\frequencies_historico_categorizado.parquet",index=False)

In [100]:
df_routes_categorizado.to_parquet(r"dados\rotas_historico_categorizado.parquet",index=False)